<a href="https://colab.research.google.com/github/Mojisibe/Inflation-rate/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("Hello world!")

Hello world!


In [ ]:
importimport pymc as pm
import numpy as np
import pandas as pd
import arviz as az
import matplotlib.pyplot as plt

def ordinal_predictor_binary_outcome_model_b02(predictor, outcome):
    """
    PyMC model with a binary outcome and an ordinal predictor with a normal random walk constraint,
    with fluctuations around an estimated effect.
    Also includes PPCs, WAIC, and BIC calculation.
    """
    coords = {"obs": np.arange(len(outcome))} # Define the 'obs' dimension using coords
    with pm.Model(coords=coords) as model: # Pass the coords to the model
        # Priors
        intercept = pm.Normal("intercept", mu=0, sigma=10)
        effect = pm.Normal("effect", mu=0, sigma=2)
        sd_fluctuation = pm.HalfCauchy("sd_fluctuation", beta=2)

        # Level effects differences
        level_effects_diff = pm.Normal("level_effects_diff", mu=effect, sigma=sd_fluctuation, shape=4)

        # Cumulative level effects
        level_effects = pm.Deterministic("level_effects", pm.math.concatenate([[0], pm.math.cumsum(level_effects_diff)]))

        # Linear predictor
        logit_p = intercept + level_effects[predictor - 1]
        p = pm.Deterministic("p", pm.math.sigmoid(logit_p))

        # Likelihood
        y_obs = pm.Bernoulli("y_obs", p=p, observed=outcome, dims="obs") # Use dims="obs" for y_obs

        # Posterior predictive samples
        y_pred = pm.Bernoulli("y_pred", p=p, observed=None, dims="obs") # Use dims="obs" for y_pred


        # Sampling
        idata = pm.sample(2000, tune=1000, return_inferencedata=True)
        idata.extend(pm.sample_posterior_predictive(idata))

        #WAIC
        waic = az.waic(idata)

        #LOO
        loo = az.loo(idata)

        #BIC calculation.
        n_params = len(model.free_RVs)
        log_likelihood = pm.log_likelihood("y_obs", idata)
        bic = -2 * np.mean(log_likelihood.y_obs) + n_params * np.log(len(outcome))

    return model, idata, waic, loo, bic # Removed pm.set_data as it was no longer needed.

In [ ]:
# Main Execution
test_data = generate_test_data()

# Run B02 Model
print("\nResults for B02 Model:")
idata_b02, waic_b02, loo_b02, bic_b02 = run_and_summarize(test_data, ordinal_predictor_binary_outcome_model_b02)


Results for B02 Model:


Output()

ERROR:pymc.stats.convergence:There were 33 divergences after tuning. Increase `target_accept` or reparameterize.


Output()

TypeError: log likelihood not found in inference data object